In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libraries
import csv
import pymongo
import pandas as pd
from pathlib import Path

In [2]:
# import csv file
salary_dataset = pd.read_excel("../Datasets/Player_salary_1990-2017.xlsx")
birthplace_dataset = pd.read_csv("../Datasets/final_heatmap.csv")
yearlyBirthplace_dataset = pd.read_csv("../Datasets/bar_chart_race.csv")

In [3]:
salary_dataset = salary_dataset.rename(columns={'Player Name':'player_name','Salary in $':'salary_$','Season Start':'season_start','Season End':'season_end','Full Team Name':'team_name'})
salary_dataset = salary_dataset.drop(axis=1,columns=['Register Value'])
salary_dataset.head()

,player_name,salary_$,season_start,season_end,Team,team_name
0,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns


In [4]:
print(len(yearlyBirthplace_dataset))

14559


# Establish connection to MongoDB

In [5]:
# Establish connection to local host 27017
conn = 'mongodb://localhost:27017'

# Connect to MongoClient
client = pymongo.MongoClient(conn)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [6]:
# Begin by dropping any previous version of the MongoDB instance.
client.drop_database('nba_players_db')

In [7]:
# Create database by naming db here
db = client.nba_players_db

# Create collection by naming collection here

### Salary collection

In [8]:
# Create salary collection
salary_collection = db.player_salary_info

In [9]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
salary_dataset.reset_index(inplace=True)

salary_dict = salary_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
salary_collection.insert_many(salary_dict)

### Player WS Collection

In [10]:
# Create collection for player WS info 1976-2020
playerWS_collection = db.playerWS


In [11]:
folder = "../Data/Data_Raw"

year = 1976

counter = 0

# i = 0

for file in Path(folder).glob('*.csv'):
    df = pd.read_csv(file)
    df.drop(df[df['Tm'] == 'TOT'].index, inplace = True)
    df = df[['Player','MP','WS', 'WS/48', 'BPM']]
    df = df.groupby(['Player']).sum().reset_index()
    df['Player'] = df['Player'].str.replace('?', '')
    df['Player'] = df['Player'].str.replace('*', '')
    for index, row in df.iterrows():
        obj = {
            'year': year,
            'player': row['Player'],
            'mp': row['MP'],
            'ws': row['WS'],
            'wsper48': row['WS/48'],
            'bpm': row['BPM']
        }
    year += 1   
    counter += 1
    
    
    playerWS_collection.insert_one(obj)

#     i += 1
#     if i == 17:
#        break

In [12]:
# len(obj)
# print(counter)

### Player birthplace collection

In [13]:
player_birthplace_collection = db.playerBirthplace

In [14]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
birthplace_dataset.reset_index(inplace=True)

birthplace_dict = birthplace_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
player_birthplace_collection.insert_many(birthplace_dict)


### Birthplace with WS and location collection

In [15]:
WS_birthplace_collection = db.WSbirthplace

In [16]:
# Creating an object from the DF
for index, row in birthplace_dataset.iterrows():
    heatmap = {
        'birthplace': row['Birth_Place'],
        'win_shares': row['WS'],
        'long': row['longitude'],
        'lat': row['latitude'], 
    }
    
    WS_birthplace_collection.insert_one(heatmap)

### Birthplace by Year collection

In [17]:
birthplace_by_year_collection = db.yearlyBirthplace

In [18]:
# Use ".to_dict('records')"in combination with .csv variable to create list of dictionaries data set
yearlyBirthplace_dataset.reset_index(inplace=True)

yearlyBirthplace_dict = yearlyBirthplace_dataset.to_dict('records')

# Insert data from the 'data_dict' variable into the collection using 'collection' variable
birthplace_by_year_collection.insert_many(yearlyBirthplace_dict)


In [19]:
# # CODE TO PRINT OUT CONTENTS OF THE DATABASE
# salary_collection = db['player_salary_info']

# result = salary_collection.find( {"Player Name": "LeBron James"} )
# # result

# for doc in result:
#     print (doc)
# ======================================================
# sal = salary_collection.find()
# sal

# for x in sal:
#     print(x)

In [20]:
# After finished with analysis, drop database by uncommenting and running this code
# client.drop_database('final_testing_beds_db')

In [21]:
# <<variable name>> = db.testing_beds.find({"<<field>>":"<<value>>"}).pretty()
